In [1]:
from mlops_codex.training import MLOpsTrainingClient

In [ ]:
client = MLOpsTrainingClient()
client

In [ ]:
training = client.create_training_experiment(
    experiment_name='Teste',
    model_type='Classification',
    group='groupname'
)

In [8]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [9]:
base_path = './samples/train/'

In [10]:
df = pd.read_csv(base_path+"/dados.csv")

In [11]:
X = df.drop(columns=['target'])
y = df[["target"]]

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df["mean_radius"], df["mean_texture"])

# Configurar o título do gráfico
plt.title("Relação entre mean_radius e mean_texture")

# Configurar os rótulos dos eixos
plt.xlabel("mean_radius")
plt.ylabel("mean_texture")

fig = plt.gcf()

# Exibir o gráfico
plt.show()


In [12]:
pipe = make_pipeline(SimpleImputer(), LGBMClassifier(force_col_wise=True))

In [ ]:
pipe.fit(X, y)

In [ ]:
with training.log_train(name='Teste 1', X_train=X, y_train=y) as logger:
    logger.save_model(pipe)
    
    model_output = pd.DataFrame({"pred": pipe.predict(X), "proba": pipe.predict_proba(X)[:,1]})
    
    logger.save_model_output(model_output)

    auc = cross_val_score(pipe, X, y, cv=5, scoring="roc_auc")
    f_score = cross_val_score(pipe, X, y, cv=5, scoring="f1")
    logger.save_metric(name='auc', value=auc.mean())
    logger.save_metric(name='f1_score', value=f_score.mean())

    logger.set_python_version('3.10')